In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
from torch.cuda.amp import autocast, GradScaler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from tqdm import tqdm

In [2]:
class Dataset2class(torch.utils.data.Dataset):
    def __init__ (self, path_dir1:str, path_dir2:str):
        super().__init__()
        
        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2
        
        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))
        
    def __len__(self):
        return len(self.dir1_list) + len(self.dir2_list)
    
    def __getitem__(self, idx):
        
        if idx < len(self.dir1_list):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        
        else:
            class_id = 1
            idx -= len(self.dir1_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])
        
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1)) # Первое место каналы, вторые высота, третье ширина
        
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor([class_id])
        
        return {'img': t_img, 'label': t_class_id}

In [3]:
Train_ds_cat_dogs = Dataset2class('C:\DATA\Computer Vision\Cats_vs_dogs\Train\dogs', 
                                  'C:\DATA\Computer Vision\Cats_vs_dogs\Train\cats')

Test_ds_cat_dogs = Dataset2class ('C:\DATA\Computer Vision\Cats_vs_dogs\Test\dogs', 
                                  'C:\DATA\Computer Vision\Cats_vs_dogs\Test\cats')

In [4]:
train_loader = torch.utils.data.DataLoader(Train_ds_cat_dogs, 
                                           shuffle=True, 
                                           batch_size=15,
                                           drop_last=True)

test_loader = torch.utils.data.DataLoader(Test_ds_cat_dogs, 
                                          shuffle=True, 
                                          batch_size=15,
                                          drop_last=False)

In [5]:
class VGG16 (nn.Module):
    
    def __init__(self, out_nc):
        
        super().__init__()
        
        self.activation = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(2, 2)
        
        
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.flat = nn.Flatten()
        
        self.fc1 = nn.Linear(25088, 4096) # 7*7*512
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, out_nc)
        
        
    def forward(self, x):
        
        out = self.conv1_1(x)
        out = self.activation(out)
        out = self.conv1_2(out)
        out = self.activation(out)
        out = self.maxpool(out)
        
        out = self.conv2_1(out)
        out = self.activation(out)
        out = self.conv2_2(out)
        out = self.maxpool(out)
        
        out = self.conv3_1(out)
        out = self.activation(out)
        out = self.conv3_2(out)
        out = self.activation(out)
        out = self.conv3_3(out)
        out = self.maxpool(out)
        
        out = self.conv4_1(out)
        out = self.activation(out)
        out = self.conv4_2(out)
        out = self.activation(out)
        out = self.conv4_3(out)
        out = self.maxpool(out)
        
        out = self.conv5_1(out)
        out = self.activation(out)
        #out = self.conv5_2(out)
        #out = self.activation(out)
        #out = self.conv5_3(out)
        out = self.maxpool(out)
        
        out = self.flat(out)
        out = self.fc1(out)
        out = self.activation(out)
        out = self.fc2(out)
        out = self.activation(out)
        out = self.fc3(out)
        
        return out

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
def accuracy(pred, label):
    answer = (torch.sigmoid(pred.detach()).numpy() > 0.5) == (label.numpy() > 0.5)
    return answer.mean()

In [8]:
#device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cuda' #if torch.cuda.is_available() else 'cpu'
model = VGG16(1) # ConvNet()
model = model.to(device)
count_parameters(model)

134264641

In [11]:
use_amp = True
scaler = torch.cuda.amp.GradScaler()

In [15]:
torch.backends.cudnn.benchmark = True
#torch.backends.cudnn.deterministic = True #дают одинаковую повторяемость обучения но будет медленнее

In [16]:
epochs = 7

loss_fn = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        
        img, label = sample['img'], sample['label']
        img = img.to(device)
        label = label.float().to(device)
        optimizer.zero_grad()
        
        with autocast (use_amp):
            #label = F.one_hot(label, 2).float()
            pred = model(img)
            #print(pred.shape, label.shape)
            loss = loss_fn(pred, label)

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item
        
        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))

  7%|█████▎                                                                           | 35/533 [00:06<01:30,  5.48it/s]


KeyboardInterrupt: 

In [2]:
torch.cuda.is_available()

True